In [1]:
%load_ext nb_black

%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [3]:
import os
import numpy as np
import pandas as pd
import spotipy
import yaml
from spotipy.oauth2 import SpotifyClientCredentials


<IPython.core.display.Javascript object>

## Download data from Spotify API

In [4]:
# Read the credentials from credentials.yml
with open("../credentials.yml") as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

# Set up the Spotify API
client_id = credentials["client_id"]  # YOUR_CLIENT_ID
client_secret = credentials["client_secret"]  # YOUR_CLIENT_SECRET
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Set global variables  
os.environ["SPOTIPY_CLIENT_ID"] = client_id
os.environ["SPOTIPY_CLIENT_SECRET"] = client_secret
os.environ["SPOTIPY_REDIRECT_URI"] = "http://localhost:8090/callback"

<IPython.core.display.Javascript object>

## Define useful functions

In [5]:
# my spotify username and playlist ids 
# on playlist page, click on "..." -> then on "Share" -> then "Copy Spotify URI"
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

<IPython.core.display.Javascript object>

## Play with the API


<div class="alert alert-block alert-success">
<b><font color='green'>Explore:</font></b> User data
</div>

In [6]:
user = sp.user('arianedalens')
print(user)

{'display_name': 'Ariane Dlns', 'external_urls': {'spotify': 'https://open.spotify.com/user/arianedalens'}, 'followers': {'href': None, 'total': 6}, 'href': 'https://api.spotify.com/v1/users/arianedalens', 'id': 'arianedalens', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab6775700000ee8542e42794651cc973c190b2fb', 'width': None}], 'type': 'user', 'uri': 'spotify:user:arianedalens'}


<IPython.core.display.Javascript object>

In [15]:
from spotipy.oauth2 import SpotifyOAuth

scope = 'playlist-read-private'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

results = sp.current_user_playlists(limit=5, offset=0)
for i, item in enumerate(results['items']):
    print("%d %s" % (i, item['name']))

0 Coline's picks
1 Radar des sorties
2 Discover Weekly
3 Your Time Capsule
4 On Repeat


<IPython.core.display.Javascript object>

In [26]:
# Get spotify singular song data from these locations
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-track
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

def getTrackFeatures(id: str):
    """
    Get the audio features for a single track
    Args:
        id: the spotify id of the track
    Returns:
        a dictionary of the audio features  
    """
    meta = sp.track(id)
    features = sp.audio_features(id)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    valence = features[0]['valence']
    time_signature = features[0]['time_signature']
    key = features[0]['key']
    mode = features[0]['mode']
    uri = features[0]['uri']

    track = [name, album, artist, release_date,
             length, popularity, acousticness,
             danceability, energy, instrumentalness, 
             liveness, loudness, speechiness, tempo, 
             valence, time_signature,
             key, mode, uri]
    return track


<IPython.core.display.Javascript object>